The next notebook its based on the [FeniCS Tutorial 1.0 documentation](https://home.simula.no/~hpl/homepage/fenics-tutorial/release-1.0-nonabla/webm/fundamentals.html)

We are trying to solve the diferential equation
$$ -\nabla^2 U(y,z) = R_e \partial_x P = C ; \ y,z \text{ in } \Omega$$
$$ U(y,z) = U_0 ; \ y,z \text{ on } \partial \Omega$$
for $C$ a constant, $U_0$ a boundary value condition and $\Omega$ the space (cross section of a tube) with $\partial \Omega$ its boundary. 

This equation is called a [Poisson equation](https://en.wikipedia.org/wiki/Poisson%27s_equation) and the boundary condition called the [Dirichlet boundary condition](https://en.wikipedia.org/wiki/Dirichlet_boundary_condition).

Let's solve it (aproximate U) for $C = 1$, $U_0 = 0$ and $\Omega = [0,1] \times [0,1]$

The way for solving this diferential equation is trough the finite element method ([FEM](https://en.wikipedia.org/wiki/Finite_element_method)).
We need to turn the PDE problem to a variatonal problem by mutiplying the equation by a function V that we call _**test function**_.
$$ -\nabla^2 U \cdot V = C \cdot V$$


Then we integrate the equation over $\Omega$
$$ -\int_\Omega \nabla^2 U \cdot V d \Omega = \int_\Omega C \cdot V d\Omega$$

Now, by applying integration by parts on the left side (or [Green's first identity](https://en.wikipedia.org/wiki/Green%27s_identities)) we get
$$ -\int_\Omega \nabla^2 U \cdot V d\Omega = \int_\Omega \nabla U \cdot \nabla V d\Omega - \int_{\partial \Omega} V \cdot \nabla U \cdot \hat{n} \ d\partial \Omega $$

The test function is required to vanish on the parts of the boundary (for convinience) where $U$ is known, implying that $V = 0$ on the whole boundary $\partial \Omega$, deleting the second integral and letting us with
$$ \int_\Omega \nabla U \cdot \nabla V d\Omega = \int_\Omega C \cdot V d\Omega$$

This equation is suposed to hold for hold for all $V$ in some function space $\hat{T}$. The function we are trying to aproximate is called _**trial function**_, and lies in some (possibly different) function space $T$. We say that the last equation is the _[**weak form**](https://en.wikipedia.org/wiki/Weak_formulation)_ of the original boundary value problem. It's called like this because it imposes less restrictive conditions for the solution that we can find for U (doesn't have to be 2 times diferentiable).

The proper statement of our variational problem now goes as follows: Find $U \in T$ such that
$$ \int_\Omega \nabla U \cdot \nabla V d\Omega = \int_\Omega C \cdot V d\Omega \ ; \ \forall V \in \hat{T} $$
The test and trial spaces $T$ and $\hat{T}$ are in the present problem defined as
$$ \hat{T} = \{t \in H^1(\Omega): t = 0 \text{ on } \partial \Omega\}$$
$$ T = \{t \in H^1(\Omega): t = U_0 \text{ on } \partial \Omega\}$$

Here $H^1(\Omega)$ is the [Sobolev space](https://en.wikipedia.org/wiki/Sobolev_space) containing functions $V$ such that $V^2$ and $ ||\nabla V||^2$ have finite integrals over $\Omega$. The solution of the underlying PDE must lie in a function space where also the derivatives are continuous, but the Sobolev space allows functions with discontinuous derivatives. This weaker continuity requirement of $U$ in the variational statement, caused by the integration by parts, has great practical consequences when it comes to constructing finite elements.

---

It turns out to be convenient to introduce the following unified notation for linear weak forms:
$$ a(u,v) = L(v) $$
In the present problem we have that
$$ a(U,V) = \int_\Omega \nabla U \cdot \nabla V d\Omega $$ 
$$ L(V) = \int_\Omega C \cdot V d\Omega$$

From the mathematics literature, $a(U,V)$ is known as a [_bilinear form_](https://en.wikipedia.org/wiki/Bilinear_form) and $L(V)$ as a [_linear form_](https://en.wikipedia.org/wiki/Linear_form).

---

*DISCRETIZATION*

To solve the equation numerically, we need to transform the continuous variational problem to a discrete variational problem. This is done by introducing _finite-dimensional_ test and trial spaces, often denoted as $\hat{T_h} \subset \hat{T}$ and $T_h \subset T$ (discrete spaces). The discrete variational problem reads: Find $U_h \in T_h \subset T$ such that
$$ \int_\Omega \nabla U_h \cdot \nabla V_h \ d\Omega = \int_\Omega C \cdot V_h d\Omega \ ; \ \forall V_h \in \hat{T_h} \subset \hat{T} $$

The choice of the discrete spaces follows directly from the kind of finite elements we want to apply in our problem.


In most cases, we will introduce the PDE problem with $u$ as unknown, derive a variational equation $a(u,v) =  L(v)$ with $u \in V$ and $v \in \hat{V}$, and then simply discretize the problem by saying that we choose _finite-dimensional_ spaces for $V$ and $\hat{V}$. This restriction of $V$ implies that $u$ becomes a discrete finite element function. In practice, this means that we turn our PDE problem into a continuous variational problem, create a mesh and specify an element type, and the let $V$ correspond to this mesh and element choice. Depending upon whether $V$ is infinite or finite-dimensional, $u$ will be the exact or approximate solution.

The idea behind this method is to appriximate the functions in the trial spaces as linear combination of a finite collection of functions from the test space. The finer the triangulation of the domain, the better approximation is going to be. 

To do this, we say that we can express $U_h$ as a (linear) combination of known functions, i.e. $U_h = \sum_{i=1}^N \alpha_i U_i $, with $\alpha_i$ constants (weights) and $\{U_i\}_{i=1}^N$ a base of functions for the trial space (like Lagrange or Hermite Polynomials). This are known as _degrees of freedom_ of $U$.* It is also convinient to have this functions so that then when we have to compute the integrals via other linear equations, the matrices that dictates them are sparse. We do the same with $V_h$ and then we have the next system:

$$ 
\int_\Omega \nabla \left(\sum_{i=1}^N \alpha_i U_i \right) d\Omega \cdot \nabla \left( \sum_{j=1}^N \beta_j U_j\right) d\Omega = \int_\Omega C \cdot \sum_{j=1}^N \beta_j U_j
$$
$$
\sum_{i=1}^N \sum_{j=1}^N \alpha_i \beta_j \int_\Omega \nabla U_i \cdot \nabla U_j \ d\Omega = \sum_{j=1}^N \beta_j \int_\Omega C \cdot U_j d\Omega
$$

Lastly, we can re-write the expressions
$$
\vec{\alpha} = \begin{bmatrix} \alpha_1 \\ \alpha_2 \\ \vdots \\ \alpha_N \end{bmatrix} \ ,
\vec{\beta} = \begin{bmatrix} \beta_1 \\ \beta_2 \\ \vdots \\ \beta_N \end{bmatrix} \ ,
A_{ij} = \int_\Omega \nabla U_i \cdot \nabla U_j \ d\Omega \ ,
\vec{f} = \begin{bmatrix} C \cdot \int_\Omega U_1 d\Omega\\ C \cdot \int_\Omega U_2 d\Omega\\ \vdots \\ C \cdot\int_\Omega U_N d\Omega\end{bmatrix}
$$

And we have the matrix equation
$$
\vec{\beta}^T A \vec{\alpha} = \vec{\beta}^T \vec{f}
$$
$$
A \vec{\alpha} = \vec{f}
$$
where we want to solve for the vector of coeffitients $\vec{\alpha}$

Libraries

In [1]:
from mpi4py import MPI # Message Passing Interface.
from petsc4py import PETSc # Library for solving PDE's assosiated linear problems
# Is an standar for different libraries to function properly and with high performance
import dolfinx # FeniCSx Interface
import numpy as np # Numpy, the classic
from petsc4py.PETSc import ScalarType

import ufl
from basix.ufl import element, mixed_element # Elements for the FEM
from dolfinx import default_real_type, fem, la # FEM library with the functions required for solving through FEM (for real values)
from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
) # Functions for defining variational problems and defining mathematical functions and constants
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, LinearProblem # Functions for creating the matrix problems
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary# Mesh creating functions
from ufl import div, dx, grad, inner, TrialFunction, TestFunction # operand functions

Malla

In [2]:
n = 128 # N° of divisions on each direction of the domain (x and y)

Heart = False
Circle = False
if Heart:
    msh, cell_tags, facet_tags = dolfinx.io.gmshio.read_from_msh("Meshes/Heart_mesh.msh", MPI.COMM_SELF, gdim=2)
elif Circle:
    msh, cell_tags, facet_tags = dolfinx.io.gmshio.read_from_msh("Meshes/Circle_mesh.msh", MPI.COMM_SELF, gdim=2)
else:
    msh = create_rectangle(MPI.COMM_WORLD, [np.array([0,0]), np.array([1,1])],  [n,n],  CellType.triangle)


In [3]:
V = functionspace(msh, ("Lagrange", 1))

Identificar borde para aplicar condiciones

In [ ]:
def noslip_boundary(x):
    # Given a point x, returns True if its coordinate x[i] is near the square boundary
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) | np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0)

In [5]:
if Heart:
    facets = facet_tags.indices[facet_tags.values == 12]
elif Circle:
    facets = facet_tags.indices[facet_tags.values == 2]
else:
    facets = locate_entities_boundary(msh,1, noslip_boundary)
noslip = Function(V)
dofs = locate_dofs_topological(V, 1,facets)
bc = dirichletbc(noslip, dofs)

In [6]:
u = TrialFunction(V)
v = TestFunction(V)
c = 5
f = Constant(msh, ScalarType(c))

a = form(inner(grad(u), grad(v)) *dx)
L = form(inner(f, v) * dx)

In [7]:
A = assemble_matrix(a, bcs=[bc])
A.assemble()

b = assemble_vector(L)

# "Ajustamos" b para que se mantenga la igualad Ax = b luego de aplicarle las bcs a A
apply_lifting(b, [a], bcs=[[bc]])

# Sincroniza los "Ghost values" usados para la computación paralela
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Aplicamos las condiciones de borde a b
bc.set(b)

In [9]:
# Configuramos el solver (Krylov Subspace method)
ksp = PETSc.KSP().create(msh.comm) 
ksp.setOperators(A) # Define la matriz A del sistema

pc = ksp.getPC() # Precondicionador de ksp
pc.setType("lu") # El preocondicionador a usar (de LU)

pc.setFactorSolverType("superlu_dist") # Establece el tipo de solver para calcular el precondicionador/factorización
vel = Function(V)
ksp.solve(b, vel.x.petsc_vec)

In [10]:
from dolfinx.io import VTKFile

# Guardar el archivo de la velocidad
if Heart:
    with VTKFile(msh.comm, "Heart_poisson.vtu", "w") as vtk:
        vtk.write_function(vel)
elif Circle:
    with VTKFile(msh.comm, "Circle_poisson.vtu", "w") as vtk:
        vtk.write_function(vel)
else:
    with VTKFile(msh.comm, "Square_poisson.vtu", "w") as vtk:
        vtk.write_function(vel)